In [1]:
# only permit vertex cover here

In [2]:
canonical_order = ['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz',  'MUTAG', 'ENZYMES', 'PROTEINS',   'IMDB-BINARY', 'COLLAB']
def reorder(df, canonical_order, by='dataset', extras=['dataset','gen_n', 'gen_n_max'], secondary='gen_n'):
    df['dataset_name_order'] = df[by].map({name: i for i, name in enumerate(canonical_order)})
    if secondary is not None:
        df = df.sort_values(by=['dataset_name_order', secondary])
    else:
        df = df.sort_values(by=['dataset_name_order'])

    columnorder = [x for x in canonical_order if x in df.keys()]
    return df[extras + columnorder]

In [3]:
import os
os.listdir('/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs')

['legacy',
 'Testing',
 '230928_snapshot',
 'LiftMP_runs',
 '230924_hparam2',
 '230927_snapshot',
 '230924_hparam',
 '230924_hparam_TU_multiarch',
 '230924_hparam_TU',
 '230926_finetune_ER_runs']

print(list(os.path.join('230927_snapshot', x) for x in os.listdir('/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/230927_snapshot')))

run_folders = ['LiftMP_runs', '230927_snapshot/230925_TUsmall_GAT_VC', '230927_snapshot/230925_TUsmall_GIN_VC', 
               '230927_snapshot/230925_generated_preset_cut', '230927_snapshot/230925_TUlarge_all_cut', '230927_snapshot/230925_TUsmall_GCNN_VC',
                 '230927_snapshot/230925_generated_liftMP_VC', '230927_snapshot/230925_TUlarge_all_VC', '230927_snapshot/230925_generated_preset_VC', 
                 '230927_snapshot/230925_TUsmall_liftMP_VC', '230927_snapshot/230925_TUsmall_GatedGCNN_VC', '230927_snapshot/230925_generated_liftMP_cut']


In [4]:
import json
import pandas as pd
import numpy as np

#folder_path = '/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs'
model_list = ["/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:1861e3873b61c0138f0fade13ba831a6cb31a0e2d5717c68a20a0009aa3859fb/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:864d82fb8b5b3647c51f85219aee5730cb01098f64dc5fafe29934637b25f1f0/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:39dcb2727b0fbd50b59b960e311816caf1dbce32cc94096c070c192f530359a4/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:5d1b0989edbfea2e797072b8c2b12001f723339dfa96406758e6208905030c61/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:7eae26a6a2d778806fa220b51c7c589f35ef252023708a0e1b70f1d809fefa16/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:c259ee991d44601a24495085d478929b226e94ca5ffc2a75ab739957d94da4b0/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:9ede9cd33a4971c3246500914847936dbc9690bd4c05a450cdef0d6146b815f4/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:631e9afcf3bd0330c4f9625392cc0b183926a43b2ba3d8c5d8fbaf91c779dc90/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:44c7a07f4cfa3adea844271356ad4eb9f408bafc526faf912be9985288fd058e/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:0e1f84c7f350a2df384d2be3ccb214f0a94111255927a19fbe60c460c9117ebb/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:176039cd545a47b6cb218b27e31f75dd958f21f293303d95e97a3964c1ae558c/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:81ae831246b731c00408289a4fa29a8586fff742dbb3d6f2fea260065f5e034f/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:53efe2843e3a8f9199398958fe32799b669b22c19f054ab089fb55ebb1b7534e/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:2a0f6b1a8c8feada2d90a7777d71b5652e2a0407b78bf34545fc38f33bdfdaaf/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:83fbb622af5fbb000c97c17df538bd3bee6082a5f9786d9a4f91ef7ce1045b28/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:034bbc1b5d117369cee302d1e2f6b87ba5ec5d5b4bf3a95e7fc99586f96fb415/",
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:815075705c4d1fec7ff5e2364cd541bc9cf945ca345e6568e1d5da54e795c7c8/", ]


rows = []

errored = []
dss = []
for model_folder in model_list:
    try:
        with open(os.path.join(model_folder, 'params.txt'), 'r') as f:
            model_args = json.load(f)
        if model_args['problem_type'] != 'vertex_cover':
            continue

        modeldict = model_args 
        for x in os.listdir(model_folder):
            if "@@" in x:
                # fudge...
                key = x.split("@@")[0]
                if not key.endswith('_testonly'):
                    continue
                key = key[:-9]
                
                #print(key)
                (_, score) = np.load(os.path.join(model_folder, x))

                
                dss.append(key)
                modeldict[key] = -1*score
        
        
        #losses = np.load(os.path.join(model_folder, 'valid_scores.npy'))
        #test_losses = np.load(os.path.join(model_folder, 'test_scores.npy'))
        #modeldict = model_args #{x: model_args[x] for x in params}
        #modeldict['max_valid_score'] = max(losses)
        #modeldict['max_valid_epoch'] = np.argmax(losses)
        #modeldict['scores'] = test_losses[np.argmax(losses)]
        modeldict['baseline'] = False
        rows.append(modeldict)
        
    except Exception as e:
        print(f'{e} is wrong w/ {model_folder}')
        errored.append(model_folder)

In [5]:
dss = list(set(dss))
dss

['ENZYMES', 'PROTEINS', 'MUTAG', 'IMDB-BINARY', 'COLLAB']

In [6]:
# load in baselines
baselines = pd.read_csv('vc_baseline_scores.csv').head(3)
baselines = baselines.applymap(lambda x: x * -1 if isinstance(x, float) else x)



In [7]:
df = pd.DataFrame.from_records(rows)

In [8]:
df['gen_n_max'] = df.gen_n.apply(lambda x: x[-1] if isinstance(x,list) else "")
df.gen_n = df.gen_n.apply(lambda x: x[0] if isinstance(x,list) else "")


In [9]:
reorder(df, canonical_order)

dataset gen_n gen_n_max     MUTAG    ENZYMES   PROTEINS  \
0    BarabasiAlbert    50       100  7.736842  20.116667  27.660714   
1    BarabasiAlbert   100       200  7.736842  20.350000  26.026786   
2    BarabasiAlbert   400       500  8.052632  21.000000  26.535714   
5        ErdosRenyi    50       100  7.736842  20.366667  28.169643   
6        ErdosRenyi   100       200  8.052632  21.516667  27.723214   
7        ErdosRenyi   400       500  7.789474  21.550000  28.598214   
11  PowerlawCluster    50       100  7.736842  20.416667  25.598214   
12  PowerlawCluster   100       200  7.842105  20.433333  27.303571   
13  PowerlawCluster   400       500  7.947368  20.633333  26.303571   
14    WattsStrogatz    50       100  7.894737  20.133333  25.464286   
15    WattsStrogatz   100       200  7.789474  20.300000  25.446429   
16    WattsStrogatz   400       500  8.052632  20.483333  25.785714   
9             MUTAG                  7.736842  20.833333  26.758929   
4           ENZYMES                  7.736842  20.600000  28.294643   
10         PROTEINS                  7.894737  20.216667  25.285714   
8       IMDB-BINARY                  7.947368  20.966667  27.062500   
3            COLLAB                  7.894737  20.350000  26.133929   

    IMDB-BINARY  COLLAB  
0         17.57  74.152  
1         16.86  69.294  
2         17.34  70.170  
5         16.86  69.066  
6         16.89  68.830  
7         16.78  68.740  
11        17.05  69.166  
12        17.01  70.200  
13        17.15  69.912  
14        17.38  70.142  
15        17.91  71.160  
16        17.12  70.156  
9         16.92  70.088  
4         16.79  68.398  
10        16.77  70.260  
8         16.76  68.034  
3         16.76  67.522

df[['dataset', 'gen_n'] + dss]

In [10]:
pd.concat([ reorder(df[['dataset', 'gen_n', 'gen_n_max'] + dss], canonical_order), baselines[['Unnamed: 0'] + dss].sort_values('Unnamed: 0')])[['Unnamed: 0'] + ['dataset', 'gen_n', 'gen_n_max'] + [x for x in canonical_order if x in dss]].to_csv('Generalizability.csv')

/tmp/ipykernel_1529064/3755298099.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dataset_name_order'] = df[by].map({name: i for i, name in enumerate(canonical_order)})
